In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

In [ ]:
# test connetion
!ls '/content/gdrive/My Drive'

**IMPORTANT:  make sure to run all programs before comaprign the restults.**

**Compare methods agaisnt each other with all test images**

In [ ]:
import os
from collections import defaultdict, Counter

# Define a function to parse a netlist file into a component-to-nodes mapping and a node-to-components mapping
def parse_netlist(file_content):
    component_to_nodes = {}
    node_to_components = defaultdict(list)  # Change to list to preserve component types

    for line in file_content.strip().split("\n"):
        parts = line.split()
        component = parts[0]
        connections = list(map(int, parts[1:]))
        component_to_nodes[component] = connections
        for node in connections:
            node_to_components[node].append(component)

    return component_to_nodes, node_to_components

# Count component types connected to each node
def count_component_types(node_to_components):
    node_component_type_counts = {}
    for node, components in node_to_components.items():
        component_type_counts = Counter(comp.split('_')[0] for comp in components)  # Extract component type
        node_component_type_counts[node] = component_type_counts
    return node_component_type_counts

# Function to compare node equivalence based on counts of connected component types
def find_equivalent_nodes_by_type(correct_node_map, generated_node_map):
    correct_to_generated_mapping = {}
    generated_to_correct_mapping = {}

    # Check each node in the correct netlist
    for correct_node, correct_counts in correct_node_map.items():
        found_match = False
        for generated_node, generated_counts in generated_node_map.items():
            if correct_counts == generated_counts:
                correct_to_generated_mapping[correct_node] = generated_node
                generated_to_correct_mapping[generated_node] = correct_node
                found_match = True
                break

        if not found_match:
            correct_to_generated_mapping[correct_node] = None  # No match found

    return correct_to_generated_mapping, generated_to_correct_mapping

# Function to check the netlist equivalence
def check_netlist_equivalence_by_type(correct_node_map, generated_node_map):
    correct_to_generated, generated_to_correct = find_equivalent_nodes_by_type(correct_node_map, generated_node_map)

    is_equivalent = all(node is not None for node in correct_to_generated.values())
    return is_equivalent, correct_to_generated, generated_to_correct

# Function to calculate performance metrics
def calculate_performance_by_type(correct_node_map, generated_node_map, correct_to_generated_mapping):
    total_correct_nodes = len(correct_node_map)
    matched_nodes = sum(1 for node in correct_to_generated_mapping.values() if node is not None)
    accuracy = (matched_nodes / total_correct_nodes) * 100 if total_correct_nodes > 0 else 0
    return accuracy, matched_nodes, total_correct_nodes

# Function to process a folder and calculate performance metrics
def process_folder(folder_path, correct_results_folder):
    total_accuracy = 0
    total_matched_nodes = 0
    total_correct_nodes = 0
    num_circuits = 0

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):  # Only process text files
            generated_file_path = os.path.join(folder_path, filename)
            correct_file_path = os.path.join(correct_results_folder, filename)

            if not os.path.exists(correct_file_path):
                print(f"Skipping {filename}: Correct file not found.")
                continue

            # Read the netlist files
            with open(correct_file_path) as f:
                correct_netlist_content = f.read()

            with open(generated_file_path) as f:
                generated_netlist_content = f.read()

            # Parse the netlist files
            _, correct_node_to_components = parse_netlist(correct_netlist_content)
            _, generated_node_to_components = parse_netlist(generated_netlist_content)

            # Count component types connected to each node
            correct_node_type_counts = count_component_types(correct_node_to_components)
            generated_node_type_counts = count_component_types(generated_node_to_components)

            # Check equivalence
            is_equivalent, correct_to_generated, generated_to_correct = check_netlist_equivalence_by_type(correct_node_type_counts, generated_node_type_counts)

            # Calculate performance metrics
            accuracy, matched_nodes, total_nodes = calculate_performance_by_type(correct_node_type_counts, generated_node_type_counts, correct_to_generated)

            # Update totals
            total_accuracy += accuracy
            total_matched_nodes += matched_nodes
            total_correct_nodes += total_nodes
            num_circuits += 1

    # Calculate overall performance
    avg_accuracy = total_accuracy / num_circuits if num_circuits > 0 else 0
    return avg_accuracy, total_matched_nodes, total_correct_nodes, num_circuits

# Specify folder paths
main_path = '/content/gdrive/My Drive/Netlist generator/Program test/Netlist generator algorithm test/'
method_1_folder = main_path + 'Method 1/Test results/'
method_2_folder = main_path + 'Method 2/Test results/'
correct_results_folder = main_path + 'Correct netlist results/'

# Process Method 1 and Method 2 folders
print("Processing Method 1...")
method_1_metrics = process_folder(method_1_folder, correct_results_folder)
print("Processing Method 2...")
method_2_metrics = process_folder(method_2_folder, correct_results_folder)

# Display results
print("\nOverall Performance Metrics for Method 1:")
print(f"Average Accuracy: {method_1_metrics[0]:.2f}%")
print(f"Total Matched Nodes: {method_1_metrics[1]}")
print(f"Total Nodes in All Correct Netlists: {method_1_metrics[2]}")

print("\nOverall Performance Metrics for Method 2:")
print(f"Average Accuracy: {method_2_metrics[0]:.2f}%")
print(f"Total Matched Nodes: {method_2_metrics[1]}")
print(f"Total Nodes in All Correct Netlists: {method_2_metrics[2]}")

# Compare results
print("\nComparison:")
if method_1_metrics[0] > method_2_metrics[0]:
    print("Method 1 performed better in terms of accuracy.")
elif method_1_metrics[0] < method_2_metrics[0]:
    print("Method 2 performed better in terms of accuracy.")
else:
    print("Both methods performed equally in terms of accuracy.")


**Fully test one method with all test images**

In [ ]:
import os
from collections import defaultdict, Counter

# Define a function to parse a netlist file into a component-to-nodes mapping and a node-to-components mapping
def parse_netlist(file_content):
    component_to_nodes = {}
    node_to_components = defaultdict(list)  # Change to list to preserve component types

    for line in file_content.strip().split("\n"):
        parts = line.split()
        component = parts[0]
        connections = list(map(int, parts[1:]))
        component_to_nodes[component] = connections
        for node in connections:
            node_to_components[node].append(component)

    return component_to_nodes, node_to_components

# Count component types connected to each node
def count_component_types(node_to_components):
    node_component_type_counts = {}
    for node, components in node_to_components.items():
        component_type_counts = Counter(comp.split('_')[0] for comp in components)  # Extract component type
        node_component_type_counts[node] = component_type_counts
    return node_component_type_counts

# Function to compare node equivalence based on counts of connected component types
def find_equivalent_nodes_by_type(correct_node_map, generated_node_map):
    correct_to_generated_mapping = {}
    generated_to_correct_mapping = {}

    # Check each node in the correct netlist
    for correct_node, correct_counts in correct_node_map.items():
        found_match = False
        for generated_node, generated_counts in generated_node_map.items():
            if correct_counts == generated_counts:
                correct_to_generated_mapping[correct_node] = generated_node
                generated_to_correct_mapping[generated_node] = correct_node
                found_match = True
                break

        if not found_match:
            correct_to_generated_mapping[correct_node] = None  # No match found

    return correct_to_generated_mapping, generated_to_correct_mapping

# Function to check the netlist equivalence
def check_netlist_equivalence_by_type(correct_node_map, generated_node_map):
    correct_to_generated, generated_to_correct = find_equivalent_nodes_by_type(correct_node_map, generated_node_map)

    is_equivalent = all(node is not None for node in correct_to_generated.values())
    return is_equivalent, correct_to_generated, generated_to_correct

# Function to calculate performance metrics
def calculate_performance_by_type(correct_node_map, generated_node_map, correct_to_generated_mapping):
    total_correct_nodes = len(correct_node_map)
    matched_nodes = sum(1 for node in correct_to_generated_mapping.values() if node is not None)
    accuracy = (matched_nodes / total_correct_nodes) * 100 if total_correct_nodes > 0 else 0
    return accuracy, matched_nodes, total_correct_nodes

# Specify folder paths
test_circuit = 'GND_test'
main_path = '/content/gdrive/My Drive/Netlist generator/Program test/Netlist generator algorithm test/'
method_2_test_results_folder = main_path + 'Method 2/Test results/'
correct_results_folder = main_path + 'Correct netlist results/'

# Initialize totals for final performance metrics
total_accuracy = 0
total_matched_nodes = 0
total_correct_nodes = 0
num_circuits = 0

# Iterate over all files in the "Method 2/Test results" folder
for filename in os.listdir(method_2_test_results_folder):
    if filename.endswith(".txt"):  # Only process text files
        generated_file_path = os.path.join(method_2_test_results_folder, filename)
        correct_file_path = os.path.join(correct_results_folder, filename)

        if not os.path.exists(correct_file_path):
            print(f"Skipping {filename}: Correct file not found.")
            continue

        # Read the netlist files
        with open(correct_file_path) as f:
            correct_netlist_content = f.read()

        with open(generated_file_path) as f:
            generated_netlist_content = f.read()

        # Parse the netlist files
        _, correct_node_to_components = parse_netlist(correct_netlist_content)
        _, generated_node_to_components = parse_netlist(generated_netlist_content)

        # Count component types connected to each node
        correct_node_type_counts = count_component_types(correct_node_to_components)
        generated_node_type_counts = count_component_types(generated_node_to_components)

        # Check equivalence
        is_equivalent, correct_to_generated, generated_to_correct = check_netlist_equivalence_by_type(correct_node_type_counts, generated_node_type_counts)

        # Calculate performance metrics
        accuracy, matched_nodes, total_nodes = calculate_performance_by_type(correct_node_type_counts, generated_node_type_counts, correct_to_generated)

        # Update totals
        total_accuracy += accuracy
        total_matched_nodes += matched_nodes
        total_correct_nodes += total_nodes
        num_circuits += 1

        # Print performance metrics for the current circuit
        print(f"Results for {filename}:")
        print(f"Performance Metrics:")
        print(f"Accuracy: {accuracy:.2f}%")
        print(f"Matched Nodes: {matched_nodes}")
        print(f"Total Nodes in Correct Netlist: {total_nodes}")
        print("")

# Calculate and print final performance metrics
if num_circuits > 0:
    avg_accuracy = total_accuracy / num_circuits
    print("Overall Performance Metrics:")
    print(f"Average Accuracy: {avg_accuracy:.2f}%")
    print(f"Total Matched Nodes: {total_matched_nodes}")
    print(f"Total Nodes in All Correct Netlists: {total_correct_nodes}")
else:
    print("No circuits were processed.")


**test one method on one image with more details for debugging**

In [ ]:
from collections import defaultdict, Counter

# Define a function to parse a netlist file into a component-to-nodes mapping and a node-to-components mapping
def parse_netlist(file_content):
    component_to_nodes = {}
    node_to_components = defaultdict(list)  # Change to list to preserve component types

    for line in file_content.strip().split("\n"):
        parts = line.split()
        component = parts[0]
        connections = list(map(int, parts[1:]))
        component_to_nodes[component] = connections
        for node in connections:
            node_to_components[node].append(component)

    return component_to_nodes, node_to_components

# Count component types connected to each node
def count_component_types(node_to_components):
    node_component_type_counts = {}
    for node, components in node_to_components.items():
        component_type_counts = Counter(comp.split('_')[0] for comp in components)  # Extract component type
        node_component_type_counts[node] = component_type_counts
    return node_component_type_counts

# Function to compare node equivalence based on counts of connected component types
def find_equivalent_nodes_by_type(correct_node_map, generated_node_map):
    correct_to_generated_mapping = {}
    generated_to_correct_mapping = {}

    # Check each node in the correct netlist
    for correct_node, correct_counts in correct_node_map.items():
        found_match = False
        for generated_node, generated_counts in generated_node_map.items():
            if correct_counts == generated_counts:
                correct_to_generated_mapping[correct_node] = generated_node
                generated_to_correct_mapping[generated_node] = correct_node
                found_match = True
                break

        if not found_match:
            correct_to_generated_mapping[correct_node] = None  # No match found

    return correct_to_generated_mapping, generated_to_correct_mapping

# Function to check the netlist equivalence
def check_netlist_equivalence_by_type(correct_node_map, generated_node_map):
    correct_to_generated, generated_to_correct = find_equivalent_nodes_by_type(correct_node_map, generated_node_map)

    is_equivalent = all(node is not None for node in correct_to_generated.values())
    return is_equivalent, correct_to_generated, generated_to_correct

# Function to calculate performance metrics
def calculate_performance_by_type(correct_node_map, generated_node_map, correct_to_generated_mapping):
    total_correct_nodes = len(correct_node_map)
    matched_nodes = sum(1 for node in correct_to_generated_mapping.values() if node is not None)
    accuracy = (matched_nodes / total_correct_nodes) * 100 if total_correct_nodes > 0 else 0
    return accuracy, matched_nodes, total_correct_nodes

# Specify file paths
test_circuit = 'GND_test'
main_path = '/content/gdrive/My Drive/Netlist generator/Program test/Netlist generator algorithm test/'
method_2_test_results_path = main_path + 'Method 2/Test results/' + test_circuit + '.txt'

generated_file_path = method_2_test_results_path
correct_file_path = '/content/gdrive/My Drive/Netlist generator/Program test/Netlist generator algorithm test/Correct netlist results/' + test_circuit + '.txt'

# Read the netlist files
with open(correct_file_path) as f:
    correct_netlist_content = f.read()

with open(generated_file_path) as f:
    generated_netlist_content = f.read()

# Parse the netlist files
_, correct_node_to_components = parse_netlist(correct_netlist_content)
_, generated_node_to_components = parse_netlist(generated_netlist_content)

# Count component types connected to each node
correct_node_type_counts = count_component_types(correct_node_to_components)
generated_node_type_counts = count_component_types(generated_node_to_components)

# Check equivalence
is_equivalent, correct_to_generated, generated_to_correct = check_netlist_equivalence_by_type(correct_node_type_counts, generated_node_type_counts)

# Calculate performance metrics
accuracy, matched_nodes, total_nodes = calculate_performance_by_type(correct_node_type_counts, generated_node_type_counts, correct_to_generated)

# Print results
print("Node Mapping from Correct to Generated:")
for correct_node, generated_node in correct_to_generated.items():
    print(f"Node {correct_node} in Correct File -> Node {generated_node} in Generated File")

print("\nNode Mapping from Generated to Correct:")
for generated_node, correct_node in generated_to_correct.items():
    print(f"Node {generated_node} in Generated File -> Node {correct_node} in Correct File")

print("\nNetlist Equivalence:")
if is_equivalent:
    print("The netlists are equivalent.")
else:
    print("The netlists are not equivalent.")

# Print performance metrics
print("\nPerformance Metrics:")
print(f"Accuracy: {accuracy:.2f}%")
print(f"Matched Nodes: {matched_nodes}")
print(f"Total Nodes in Correct Netlist: {total_nodes}")

print("\nNode Component Type Counts (Correct File):")
for node, counts in correct_node_type_counts.items():
    print(f"Node {node}: {dict(counts)}")

print("\nNode Component Type Counts (Generated File):")
for node, counts in generated_node_type_counts.items():
    print(f"Node {node}: {dict(counts)}")
